# E2LSH vs SimHash 比較実験

## 目的

SimHash（角度ベース）とE2LSH（ユークリッド距離ベース）のRecall@10を比較する。

## 背景

- **SimHash**: h(v) = sign(a·v) - 角度（コサイン類似度）ベース
- **E2LSH**: h(v) = floor((a·v + b) / w) - ユークリッド距離ベース

Embeddingベクトルは通常L2正規化されているため、コサイン類似度とユークリッド距離は関連するが、
LSHの衝突特性は異なる可能性がある。

In [1]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
from numpy.typing import NDArray

In [2]:
from lsh import SimHashGenerator, hamming_distance, chunk_hash
from e2lsh import E2LSHHasher, E2LSHIndex, hash_distance

## 1. データ読み込み

前回の実験でチャンクなしで最も高いRecallを達成したJina-v3のEmbeddingを使用。

In [3]:
# Jina-v3のEmbedding（最も良い結果）を読み込み
df = pd.read_parquet('../data/embeddings_jina_v3.parquet')
print(f"データ件数: {len(df)}")
print(f"カラム: {df.columns.tolist()}")

データ件数: 10000
カラム: ['id', 'text', 'vector', 'simhash', 'lsh_chunks']


In [4]:
# Embeddingベクトルを抽出
embeddings = np.stack(df['vector'].values).astype(np.float32)
print(f"Embedding shape: {embeddings.shape}")

# L2正規化を確認
norms = np.linalg.norm(embeddings, axis=1)
print(f"Norm range: [{norms.min():.4f}, {norms.max():.4f}]")

Embedding shape: (10000, 1024)
Norm range: [0.9962, 1.0039]


## 2. 評価用クエリとGround Truthの準備

In [5]:
# クエリとして100件をサンプリング
np.random.seed(42)
num_queries = 100
query_indices = np.random.choice(len(embeddings), num_queries, replace=False)

# Ground Truth: 各クエリに対するコサイン類似度Top10
def compute_ground_truth(embeddings: NDArray, query_indices: NDArray, top_k: int = 10):
    """コサイン類似度ベースのGround Truthを計算"""
    ground_truth = {}
    
    for q_idx in query_indices:
        query = embeddings[q_idx]
        # コサイン類似度（L2正規化済みなので内積 = コサイン類似度）
        similarities = embeddings @ query
        # 自分自身を除外するために-infを設定
        similarities[q_idx] = -np.inf
        # Top-kのインデックス
        top_indices = np.argsort(similarities)[::-1][:top_k]
        ground_truth[q_idx] = set(top_indices)
    
    return ground_truth

ground_truth = compute_ground_truth(embeddings, query_indices)
print(f"クエリ数: {len(ground_truth)}")

クエリ数: 100


## 3. SimHashの評価

ベースラインとして既存のSimHashを評価。

In [6]:
def evaluate_simhash(embeddings: NDArray, query_indices: NDArray, ground_truth: dict,
                     hash_bits: int = 128, num_chunks: int = 0, step2_top_n: int = 100):
    """SimHashのRecall@10を評価
    
    Args:
        num_chunks: 0の場合はチャンクなし（全ハッシュ一致のみ候補）
    """
    hasher = SimHashGenerator(dim=embeddings.shape[1], hash_bits=hash_bits, seed=42)
    
    # 全ベクトルをハッシュ化
    all_hashes = hasher.hash_batch(embeddings)
    
    recalls = []
    candidates_counts = []
    
    for q_idx in query_indices:
        query_hash = all_hashes[q_idx]
        
        if num_chunks == 0:
            # チャンクなし: ハミング距離でソートして上位を取得
            distances = [(i, hamming_distance(query_hash, h)) for i, h in enumerate(all_hashes) if i != q_idx]
            distances.sort(key=lambda x: x[1])
            candidates = [i for i, _ in distances[:step2_top_n]]
        else:
            # チャンク分割: 少なくとも1つのチャンクが一致する候補を収集
            query_chunks = set(chunk_hash(query_hash, num_chunks))
            candidates = set()
            
            for i, h in enumerate(all_hashes):
                if i == q_idx:
                    continue
                doc_chunks = set(chunk_hash(h, num_chunks))
                if query_chunks & doc_chunks:  # 交差があれば候補
                    candidates.add(i)
            
            # ハミング距離でソート
            if candidates:
                distances = [(i, hamming_distance(query_hash, all_hashes[i])) for i in candidates]
                distances.sort(key=lambda x: x[1])
                candidates = [i for i, _ in distances[:step2_top_n]]
            else:
                candidates = []
        
        candidates_counts.append(len(candidates))
        
        # Recall@10を計算
        gt = ground_truth[q_idx]
        retrieved = set(candidates[:10])
        recall = len(gt & retrieved) / len(gt)
        recalls.append(recall)
    
    return {
        'recall_mean': np.mean(recalls),
        'recall_std': np.std(recalls),
        'candidates_mean': np.mean(candidates_counts),
    }

In [7]:
# SimHash: チャンクなし
result_simhash_nochunk = evaluate_simhash(embeddings, query_indices, ground_truth, num_chunks=0)
print(f"SimHash (No Chunk): Recall@10 = {result_simhash_nochunk['recall_mean']:.3f} ± {result_simhash_nochunk['recall_std']:.3f}")

SimHash (No Chunk): Recall@10 = 0.258 ± 0.208


In [8]:
# SimHash: chunks=8
result_simhash_c8 = evaluate_simhash(embeddings, query_indices, ground_truth, num_chunks=8)
print(f"SimHash (chunks=8): Recall@10 = {result_simhash_c8['recall_mean']:.3f} ± {result_simhash_c8['recall_std']:.3f}")

SimHash (chunks=8): Recall@10 = 0.080 ± 0.156


## 4. E2LSHの評価

E2LSHはパラメータが多い:
- **w (bucket width)**: バケット幅。大きいほど衝突しやすい
- **k**: テーブルあたりのハッシュ関数数。大きいほど厳密
- **L (num_tables)**: テーブル数。多いほど再現率向上

In [9]:
def evaluate_e2lsh(embeddings: NDArray, query_indices: NDArray, ground_truth: dict,
                   w: float = 4.0, k: int = 8, num_tables: int = 1, step2_top_n: int = 100):
    """E2LSHのRecall@10を評価"""
    dim = embeddings.shape[1]
    hasher = E2LSHHasher(dim=dim, w=w, k=k, num_tables=num_tables, seed=42)
    
    # インデックス構築
    index = E2LSHIndex(hasher)
    index.build(embeddings)
    
    recalls = []
    candidates_counts = []
    
    for q_idx in query_indices:
        query = embeddings[q_idx]
        
        # E2LSHで候補を取得
        candidates = index.query(query, top_k=step2_top_n)
        
        # 自分自身を除外
        candidates = [c for c in candidates if c != q_idx]
        candidates_counts.append(len(candidates))
        
        # Recall@10を計算
        gt = ground_truth[q_idx]
        retrieved = set(candidates[:10])
        recall = len(gt & retrieved) / len(gt)
        recalls.append(recall)
    
    return {
        'recall_mean': np.mean(recalls),
        'recall_std': np.std(recalls),
        'candidates_mean': np.mean(candidates_counts),
    }

In [10]:
# E2LSH: デフォルトパラメータ (w=4.0, k=8, L=1)
result_e2lsh_default = evaluate_e2lsh(embeddings, query_indices, ground_truth, w=4.0, k=8, num_tables=1)
print(f"E2LSH (w=4.0, k=8, L=1): Recall@10 = {result_e2lsh_default['recall_mean']:.3f} ± {result_e2lsh_default['recall_std']:.3f}")
print(f"  平均候補数: {result_e2lsh_default['candidates_mean']:.1f}")

E2LSH (w=4.0, k=8, L=1): Recall@10 = 0.227 ± 0.215
  平均候補数: 88.9


## 5. E2LSHパラメータチューニング

様々なパラメータ組み合わせを試す。

In [11]:
# パラメータグリッドサーチ
results = []

# w (bucket width): 大きいほど衝突増加
# k: ハッシュ関数数。大きいほど厳密
# L: テーブル数。多いほど再現率向上

param_grid = [
    # w variations
    {'w': 2.0, 'k': 8, 'num_tables': 1},
    {'w': 4.0, 'k': 8, 'num_tables': 1},
    {'w': 8.0, 'k': 8, 'num_tables': 1},
    {'w': 16.0, 'k': 8, 'num_tables': 1},
    # k variations
    {'w': 4.0, 'k': 4, 'num_tables': 1},
    {'w': 4.0, 'k': 16, 'num_tables': 1},
    # L (num_tables) variations
    {'w': 4.0, 'k': 8, 'num_tables': 4},
    {'w': 4.0, 'k': 8, 'num_tables': 8},
    {'w': 4.0, 'k': 8, 'num_tables': 16},
    # Combined tuning
    {'w': 8.0, 'k': 4, 'num_tables': 8},
    {'w': 8.0, 'k': 4, 'num_tables': 16},
    {'w': 16.0, 'k': 4, 'num_tables': 8},
]

print("E2LSH Parameter Search...")
for params in param_grid:
    result = evaluate_e2lsh(embeddings, query_indices, ground_truth, **params)
    result.update(params)
    results.append(result)
    print(f"w={params['w']:.1f}, k={params['k']}, L={params['num_tables']}: "
          f"Recall@10 = {result['recall_mean']:.3f}, Candidates = {result['candidates_mean']:.1f}")

E2LSH Parameter Search...
w=2.0, k=8, L=1: Recall@10 = 0.051, Candidates = 48.7


w=4.0, k=8, L=1: Recall@10 = 0.227, Candidates = 88.9


w=8.0, k=8, L=1: Recall@10 = 0.374, Candidates = 97.7


w=16.0, k=8, L=1: Recall@10 = 0.509, Candidates = 99.0


w=4.0, k=4, L=1: Recall@10 = 0.504, Candidates = 98.2


...

w=4.0, k=8, L=8: Recall@10 = 0.890, Candidates = 99.0


w=4.0, k=8, L=16: Recall@10 = 0.964, Candidates = 99.0


w=8.0, k=4, L=8: Recall@10 = 0.975, Candidates = 99.0


w=8.0, k=4, L=16: Recall@10 = 0.975, Candidates = 99.0


w=16.0, k=4, L=8: Recall@10 = 0.975, Candidates = 99.0


In [12]:
# 結果をDataFrameで整理
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('recall_mean', ascending=False)
print("\n=== E2LSH パラメータ比較結果 ===")
print(results_df[['w', 'k', 'num_tables', 'recall_mean', 'recall_std', 'candidates_mean']].to_string(index=False))


=== E2LSH パラメータ比較結果 ===
   w  k  num_tables  recall_mean  recall_std  candidates_mean
16.0  4           8        0.975    0.045552            99.00
 8.0  4           8        0.975    0.045552            99.00
 8.0  4          16        0.975    0.045552            99.00
 4.0  8          16        0.964    0.055714            99.00
 4.0  8           8        0.890    0.125300            99.00
 4.0  8           4        0.718    0.231249            99.00
16.0  8           1        0.509    0.249838            99.00
 4.0  4           1        0.504    0.290833            98.24
 8.0  8           1        0.374    0.231353            97.66
 4.0  8           1        0.227    0.215339            88.91
 4.0 16           1        0.153    0.186791            75.68
 2.0  8           1        0.051    0.098484            48.74


## 6. 最適E2LSH vs SimHash 最終比較

In [13]:
# 最適なE2LSHパラメータで評価
best_params = results_df.iloc[0]
print(f"Best E2LSH params: w={best_params['w']}, k={int(best_params['k'])}, L={int(best_params['num_tables'])}")

result_e2lsh_best = evaluate_e2lsh(
    embeddings, query_indices, ground_truth,
    w=best_params['w'],
    k=int(best_params['k']),
    num_tables=int(best_params['num_tables'])
)

Best E2LSH params: w=16.0, k=4, L=8


In [14]:
print("\n" + "="*60)
print("          SimHash vs E2LSH 比較結果 (Jina-v3)")
print("="*60)
print(f"\n{'手法':<30} {'Recall@10':>10} {'平均候補数':>12}")
print("-"*54)
print(f"{'SimHash (No Chunk)':<30} {result_simhash_nochunk['recall_mean']:>10.3f} {result_simhash_nochunk['candidates_mean']:>12.1f}")
print(f"{'SimHash (chunks=8)':<30} {result_simhash_c8['recall_mean']:>10.3f} {result_simhash_c8['candidates_mean']:>12.1f}")
print(f"{'E2LSH (default: w=4, k=8, L=1)':<30} {result_e2lsh_default['recall_mean']:>10.3f} {result_e2lsh_default['candidates_mean']:>12.1f}")
print(f"{'E2LSH (best params)':<30} {result_e2lsh_best['recall_mean']:>10.3f} {result_e2lsh_best['candidates_mean']:>12.1f}")
print("="*60)


          SimHash vs E2LSH 比較結果 (Jina-v3)

手法                              Recall@10        平均候補数
------------------------------------------------------
SimHash (No Chunk)                  0.258        100.0
SimHash (chunks=8)                  0.080         24.3
E2LSH (default: w=4, k=8, L=1)      0.227         88.9
E2LSH (best params)                 0.975         99.0


## 7. 他のモデルでも検証

In [15]:
def evaluate_all_methods(model_name: str, embeddings: NDArray, query_indices: NDArray, ground_truth: dict):
    """全手法を評価"""
    results = {}
    
    # SimHash (No Chunk)
    r = evaluate_simhash(embeddings, query_indices, ground_truth, num_chunks=0)
    results['SimHash_NoChunk'] = r['recall_mean']
    
    # SimHash (chunks=8)
    r = evaluate_simhash(embeddings, query_indices, ground_truth, num_chunks=8)
    results['SimHash_C8'] = r['recall_mean']
    
    # E2LSH (default)
    r = evaluate_e2lsh(embeddings, query_indices, ground_truth, w=4.0, k=8, num_tables=1)
    results['E2LSH_Default'] = r['recall_mean']
    
    # E2LSH (tuned: より多くのテーブル)
    r = evaluate_e2lsh(embeddings, query_indices, ground_truth, w=8.0, k=4, num_tables=16)
    results['E2LSH_Tuned'] = r['recall_mean']
    
    return results

In [16]:
# 全モデルで評価
all_results = {}

models = [
    ('Jina-v3', '../data/embeddings_jina_v3.parquet'),
    ('BGE-M3', '../data/embeddings_bge_m3.parquet'),
    ('E5-large', '../data/embeddings_e5_large.parquet'),
]

for model_name, path in models:
    print(f"\n評価中: {model_name}")
    df_model = pd.read_parquet(path)
    emb = np.stack(df_model['vector'].values).astype(np.float32)
    
    # Ground Truth再計算
    gt = compute_ground_truth(emb, query_indices)
    
    # 全手法を評価
    results = evaluate_all_methods(model_name, emb, query_indices, gt)
    all_results[model_name] = results
    
    print(f"  SimHash NoChunk: {results['SimHash_NoChunk']:.3f}")
    print(f"  SimHash C8: {results['SimHash_C8']:.3f}")
    print(f"  E2LSH Default: {results['E2LSH_Default']:.3f}")
    print(f"  E2LSH Tuned: {results['E2LSH_Tuned']:.3f}")


評価中: Jina-v3


  SimHash NoChunk: 0.258
  SimHash C8: 0.080
  E2LSH Default: 0.227
  E2LSH Tuned: 0.975

評価中: BGE-M3


  SimHash NoChunk: 0.245
  SimHash C8: 0.105
  E2LSH Default: 0.238
  E2LSH Tuned: 1.000

評価中: E5-large


  SimHash NoChunk: 0.157
  SimHash C8: 0.146
  E2LSH Default: 0.196
  E2LSH Tuned: 1.000


In [17]:
# 最終比較表
comparison_df = pd.DataFrame(all_results).T
print("\n" + "="*70)
print("            SimHash vs E2LSH 全モデル比較 (Recall@10)")
print("="*70)
print(comparison_df.to_string())
print("="*70)


            SimHash vs E2LSH 全モデル比較 (Recall@10)
          SimHash_NoChunk  SimHash_C8  E2LSH_Default  E2LSH_Tuned
Jina-v3             0.258       0.080          0.227        0.975
BGE-M3              0.245       0.105          0.238        1.000
E5-large            0.157       0.146          0.196        1.000


## 8. 評価レポート

### 実験結果サマリー

#### E2LSH vs SimHash 最終比較 (Recall@10)

| モデル | SimHash NoChunk | SimHash C8 | E2LSH Default | E2LSH Tuned |
|--------|-----------------|------------|---------------|-------------|
| Jina-v3 | 0.258 | 0.080 | 0.227 | **0.975** |
| BGE-M3 | 0.245 | 0.105 | 0.238 | **1.000** |
| E5-large | 0.157 | 0.146 | 0.196 | **1.000** |

### 主要な発見

1. **E2LSH + 複数テーブル（L=8〜16）でRecall@10が0.975〜1.000を達成**
   - SimHash（最高0.258）から約4倍の改善
   - 特にBGE-M3とE5-largeでは完全なRecall（1.000）を達成

2. **E2LSHのパラメータ影響**
   - **w（バケット幅）**: 大きいほど衝突確率増加 → Recall向上
   - **k（ハッシュ関数数）**: 小さいほどRecall向上（k=4が最適）
   - **L（テーブル数）**: 最も重要。L=8〜16で飽和

3. **なぜE2LSHが有効なのか**
   - SimHash: 角度（コサイン類似度）ベース → 高次元での角度変化が小さい
   - E2LSH: ユークリッド距離ベース → 複数テーブルで網羅的にカバー
   - L2正規化されたベクトルでは `||a - b||² = 2(1 - cos(a,b))` の関係あり

4. **チャンク戦略との比較**
   - SimHash + chunks=8 は候補絞り込みが過剰（Recall=0.08〜0.15）
   - E2LSH + 複数テーブル は候補を十分確保しつつ高精度

### 結論

**E2LSH（w=8.0, k=4, L=8〜16）を採用することで、LSHフィルタリングの精度を劇的に改善できる。**

| 指標 | Before (SimHash) | After (E2LSH Tuned) | 改善率 |
|------|------------------|---------------------|--------|
| Jina-v3 Recall@10 | 0.258 | 0.975 | **+278%** |
| BGE-M3 Recall@10 | 0.245 | 1.000 | **+308%** |
| E5-large Recall@10 | 0.157 | 1.000 | **+537%** |

### 次のステップ

1. E2LSHインデックスのメモリ使用量評価
2. クエリ時間のベンチマーク
3. パイプライン統合（3段階フィルタリングへの組み込み）